In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import pandas as pd

import math
import re, os, csv, sys, time, fnmatch, shutil, glob
from os import listdir
from os.path import isfile, join
from os import chdir, getcwd, path
import json, random, argparse
import io, pickle, codecs
import networkx as nx, builtins
from shutil import copyfile
import xlrd
import xlwings as xw
import numpy as np
import seaborn as sns
import pyodbc

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.datasets.samples_generator import make_blobs
from sklearn import preprocessing
from kmodes.kmodes import KModes

from matplotlib.pyplot import figure    
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (10, 5)

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Tahoma']

C:\Users\Inteligo\anaconda3.2\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
pd.options.display.max_columns = None
pd.set_option('display.max_rows', False)
# pd.set_option('display.max_rows', None)

In [3]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'ODBC Driver 13 for SQL Server',
 'ODBC Driver 11 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [4]:
# Specifying the ODBC driver, server name, database, etc. directly
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=172.17.6.184;DATABASE=GDI_LIVE;UID=USR_DO;PWD=Inteligo2021')

# Using a DSN, but providing a password as well
# cnxn = pyodbc.connect('DSN=test;PWD=password')

# Create a cursor from the connection
cursor = cnxn.cursor()

In [5]:
qTipoIngreso = ('''SELECT *
  FROM [172.18.10.62].[DWHINTELIGO].[dbo].[Lk_TipoIngreso]''')

In [6]:
dfTipoIngreso = pd.read_sql(qTipoIngreso, cnxn)

In [7]:
# PL 51000 Interest Income por cliente, el monto en cliente cero es poco
qIngIntPL51000 = ('''SELECT TbClientes.Cod_Cliente, LEFT(TbIngreso.Id_Dia,4) AS Anho, LEFT(RIGHT(TbIngreso.Id_Dia,4),2) AS Mes, TbIngreso.*
                            FROM
                            [172.18.10.62].[DWHINTELIGO].[dbo].[Ft_Ingreso] as TbIngreso,
                            [172.18.10.62].[DWHINTELIGO].[dbo].[Lk_Cliente] as TbClientes
                            WHERE
                                (TbIngreso.Id_Cliente = TbClientes.Id_Cliente and TbIngreso.id_ComponenteIngreso = 6
                                 and LEFT(TbIngreso.Id_Dia,4) = 2020)
                            ORDER BY
                                TbClientes.Cod_Cliente ASC''')

In [8]:
dfIngIntPL51000 = pd.read_sql(qIngIntPL51000, cnxn)

In [9]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=172.17.6.184;DATABASE=GDI_LIVE;UID=USR_DO;PWD=Inteligo2021')
cursor = cnxn.cursor()

In [10]:
# PL 51997 Interest Income - Overdraft por sector
qIngIntPL51997 = ('''SELECT TbClientes.Cod_Cliente, LEFT(TbIngreso.Id_Dia,4) AS Anho, LEFT(RIGHT(TbIngreso.Id_Dia,4),2) AS Mes, TbIngreso.*
                            FROM
                            [172.18.10.62].[DWHINTELIGO].[dbo].[Ft_Ingreso] as TbIngreso,
                            [172.18.10.62].[DWHINTELIGO].[dbo].[Lk_Cliente] as TbClientes
                            WHERE
                                (TbIngreso.Id_Cliente = TbClientes.Id_Cliente and TbIngreso.id_ComponenteIngreso = 17
                                 and LEFT(TbIngreso.Id_Dia,4) = 2020)
                            ORDER BY
                                TbClientes.Cod_Cliente ASC''')

In [11]:
dfIngIntPL51997 = pd.read_sql(qIngIntPL51997, cnxn)

In [27]:
# table tblAccrAcctDrMonthly with tblAccount.Customer for pl 51997

In [31]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=172.18.11.83\itlgdwh;DATABASE=ITLGDWH;UID=acabrera;PWD=Inteligo2021')
cursor = cnxn.cursor()

In [50]:
tblAccrAcctDrMonthly = ('''SELECT *
                                  FROM tblAccrAcctDrMonthly
                                  WHERE Yr = 2020''')

In [51]:
dftblAccrAcctDrMonthly = pd.read_sql(tblAccrAcctDrMonthly, cnxn)

In [52]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=172.18.11.83\itlgdwh;DATABASE=ITLGDWH;UID=acabrera;PWD=Inteligo2021')
cursor = cnxn.cursor()

In [53]:
tblAccountMonthly = ('''SELECT *
                            FROM tblAccountMonthly
                            WHERE Yr = 2020''')

In [54]:
tblAccountMonthly = pd.read_sql(tblAccountMonthly, cnxn)

In [88]:
# PL 50997

In [94]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=172.17.6.184;DATABASE=GDI_LIVE;UID=USR_DO;PWD=Inteligo2021')
cursor = cnxn.cursor()

In [95]:
# PL 50997 Interest Income - Overdraft por sector
qIngIntPL50997 = ('''SELECT
                    TbClientes.Cod_Cliente,
                    LEFT(RIGHT(TbIngreso.Id_Dia,4),2) AS Mes,
                    SUM(TbIngreso.Ingreso)
                    FROM
                    [172.18.10.62].[DWHINTELIGO].[dbo].[Ft_Ingreso] as TbIngreso,
                    [172.18.10.62].[DWHINTELIGO].[dbo].[Lk_Cliente] as TbClientes
                    WHERE
                        (TbIngreso.Id_Cliente = TbClientes.Id_Cliente and TbIngreso.id_ComponenteIngreso = 5
                         and LEFT(TbIngreso.Id_Dia,4) = 2020)
                    GROUP BY
                        TbClientes.Cod_Cliente, LEFT(RIGHT(TbIngreso.Id_Dia,4),2)
                    ORDER BY
                        TbClientes.Cod_Cliente ASC,LEFT(RIGHT(TbIngreso.Id_Dia,4),2) ASC;''')
dfIngIntPL50997 = pd.read_sql(qIngIntPL50997, cnxn)

In [89]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=172.18.11.83\itlgdwh;DATABASE=ITLGDWH;UID=acabrera;PWD=Inteligo2021')
cursor = cnxn.cursor()

In [91]:
tblMoneyMarketMonthly = ('''SELECT *
                          FROM tblMoneyMarketMonthly
                          WHERE Yr = 2020''')
dftblMoneyMarketMonthly = pd.read_sql(tblMoneyMarketMonthly, cnxn)

### PLs grouped by costumers

In [14]:
# run from here down

###### PL ingreso por interes 51000 - Interest Income

In [15]:
dfIngIntPL51000ByCli = dfIngIntPL51000.groupby(['Cod_Cliente', 'Mes'], as_index=False).agg({'Id_Cliente': 'first','Ingreso': 'sum',})

In [16]:
dfIngIntPL51000ByCli = dfIngIntPL51000ByCli.pivot_table(index=['Cod_Cliente'], columns='Mes', values='Ingreso').reset_index()

In [17]:
dfIngIntPL51000ByCli['num_meses_ing_51000'] = dfIngIntPL51000ByCli.apply(lambda x: x.iloc[1:] .count(), axis=1)
dfIngIntPL51000ByCli['total_51000'] = dfIngIntPL51000ByCli.apply(lambda x: x.iloc[1:-1] .sum(), axis=1)

In [39]:
# dfIngIntPL51000ByCli
# dfIngIntPL51000ByCli.append(dfIngIntPL51000ByCli.sum().rename('Total'))

###### PL ingreso por intereses 51997 Interest Income - Overdraft [por sector]

In [19]:
dfIngIntPL51997ByCli = dfIngIntPL51997.groupby(['Cod_Cliente', 'Mes'], as_index=False).agg({'Id_Cliente': 'first','Ingreso': 'sum',})

In [20]:
dfIngIntPL51997ByCli = dfIngIntPL51997ByCli.pivot_table(index=['Cod_Cliente'], columns='Mes', values='Ingreso').reset_index()

In [21]:
dfIngIntPL51997ByCli['num_meses_ing_51997'] = dfIngIntPL51997ByCli.apply(lambda x: x.iloc[1:] .count(), axis=1)
dfIngIntPL51997ByCli['total_51997'] = dfIngIntPL51997ByCli.apply(lambda x: x.iloc[1:-1] .sum(), axis=1)

In [22]:
dfIngIntPL51997ByCli

Mes,Cod_Cliente,01,02,03,04,05,06,07,08,09,10,11,12,num_meses_ing_51997,total_51997
0,0,-9875.08,-5852.15,-9912.51,-3777.62,-4245.6,-6197.33,-2571.59,-4938.85,-1744.29,-2773.04,-2060.0,-5765.25,12,-59713.31


In [80]:
# dftblAccrAcctDrMonthly

In [81]:
mergeAccAccount = pd.merge(dftblAccrAcctDrMonthly, tblAccountMonthly, how='left', left_on=['Id','Mth', 'Yr', 'Day'], right_on = ['Id','Mth', 'Yr', 'Day'])

In [82]:
mergeAccAccount = mergeAccAccount[['Yr', 'Mth', 'Day', 'Id', 'Customer', 'TotalInterest', 'UnadjTotalInt']]

In [84]:
mergeAccAccount = mergeAccAccount.groupby(['Customer', 'Mth'], as_index=False).agg({'TotalInterest': 'sum',}).pivot_table(index=['Customer'], columns='Mth', values='TotalInterest').reset_index()

In [85]:
mergeAccAccount['total'] = mergeAccAccount.apply(lambda x: x.iloc[1:] .sum(), axis=1)

In [87]:
mergeAccAccount.append(mergeAccAccount.sum().rename('Total'))

Mth,Customer,1,2,3,4,5,6,7,8,9,10,11,12,total
0,10025.0,NaN,NaN,NaN,NaN,NaN,857.01,NaN,NaN,NaN,NaN,NaN,NaN,857.01
1,10026.0,NaN,NaN,0.00,30.70,29.66,40.19,NaN,NaN,NaN,NaN,NaN,NaN,100.55
2,10063.0,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN,NaN,116.17,NaN,NaN,126.17
3,10074.0,20.00,10.00,10.00,NaN,NaN,NaN,20.00,10.00,10.00,10.00,NaN,NaN,90.00
4,10096.0,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00
5,10105.0,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00
6,10134.0,NaN,NaN,NaN,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN,10.00
7,10135.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,0.0,0.00
8,10136.0,NaN,NaN,NaN,39.40,NaN,10.00,NaN,0.00,NaN,NaN,NaN,NaN,49.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
# no cuadra con monto de BI

In [46]:
# dftblAccrAcctDrMonthlyFor51997 agrupado por sector

In [23]:
groupSec = dfIngIntPL51997.groupby(['Id_Sector', 'Mes'], as_index=False).agg({'Ingreso': 'sum',})

In [24]:
groupSec = groupSec.pivot_table(index=['Id_Sector'], columns='Mes', values='Ingreso').reset_index()

In [25]:
groupSec['total_sec_51997'] = groupSec.apply(lambda x: x.iloc[1:] .sum(), axis=1)

In [26]:
groupSec.sort_values(by=['total_sec_51997'], ascending=True)

Mes,Id_Sector,01,02,03,04,05,06,07,08,09,10,11,12,total_sec_51997
2,65,-9320.17,-553.45,-649.83,NaN,-120.86,-31.49,-409.61,-1208.47,-380.30,NaN,NaN,-2407.20,-15081.38
21,130,-78.29,-381.78,-5234.79,-24.30,-10.00,-179.93,-355.51,-560.26,-54.28,-47.11,-106.72,-202.54,-7235.51
22,131,NaN,-2432.48,-953.43,-855.65,-1495.13,-379.07,-103.41,-200.24,NaN,-144.94,-551.46,-73.78,-7189.59
14,112,NaN,NaN,-207.97,-722.21,-10.00,-3137.85,NaN,-55.96,-548.81,NaN,NaN,NaN,-4682.80
0,62,-24.77,-23.84,-1362.59,-237.67,-1423.18,-177.92,-142.43,-10.00,-94.38,NaN,NaN,-622.83,-4119.61
16,116,-171.37,-2064.62,-386.01,-20.00,-43.55,-39.60,-727.02,-20.52,-19.08,-132.57,NaN,-255.90,-3880.24
1,64,NaN,NaN,-139.15,-21.70,-114.16,-706.50,-165.67,-1392.48,NaN,-10.00,NaN,-1180.87,-3730.53
18,122,-280.48,-199.91,-131.74,-420.57,NaN,NaN,NaN,NaN,-901.27,NaN,-1199.86,NaN,-3133.83
25,163,NaN,NaN,NaN,NaN,NaN,NaN,-64.29,NaN,-346.58,-1592.97,NaN,-325.59,-2329.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
